# Fine-Tuning for Porous Structures (Google Colab)

This notebook fine-tunes the v3.1 MODEL C to produce more porous, lightweight architectural structures.

## Setup Instructions
1. Upload your project files OR mount Google Drive
2. Run all setup cells
3. Run the fine-tuning

In [ ]:
# Check GPU availability
import torch
print(f'PyTorch version: {torch.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    device = torch.device('cuda')
else:
    print('WARNING: No GPU detected. Training will be slow.')
    device = torch.device('cpu')

## 1. Mount Google Drive & Setup Files

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Set your project path in Google Drive
# CHANGE THIS to match your folder location!
PROJECT_PATH = '/content/drive/MyDrive/Constraint-Based-Architectural-NCA'

# Alternative: If you uploaded as a zip, uncomment below:
# !unzip /content/drive/MyDrive/nca_project.zip -d /content/
# PROJECT_PATH = '/content/Constraint-Based-Architectural-NCA'

import os
os.chdir(PROJECT_PATH)
print(f'Working directory: {os.getcwd()}')
!ls -la

In [ ]:
# Verify required files exist
import os
from pathlib import Path

required_files = [
    'notebooks/sel/MODEL C - Copy - no change/v31_fixed_geometry.pth',
    'notebooks/sel/MODEL C - Copy - no change/config_step_b.json',
    'deploy/model_utils.py'
]

all_found = True
for f in required_files:
    exists = os.path.exists(f)
    status = 'OK' if exists else 'MISSING'
    print(f'[{status}] {f}')
    if not exists:
        all_found = False

if not all_found:
    print('\nERROR: Some files are missing. Please check your PROJECT_PATH.')
else:
    print('\nAll required files found!')

## 2. Load Model & Config

In [ ]:
import sys
import json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from pathlib import Path

# Add deploy to path for imports
sys.path.insert(0, 'deploy')

from model_utils import (
    UrbanPavilionNCA,
    UrbanSceneGenerator,
    compute_corridor_target_v31,
    LocalLegalityLoss
)

print('Imports successful!')

In [ ]:
# Paths
BASE_MODEL_DIR = Path('notebooks/sel/MODEL C - Copy - no change')
CONFIG_PATH = BASE_MODEL_DIR / 'config_step_b.json'
CHECKPOINT_PATH = BASE_MODEL_DIR / 'v31_fixed_geometry.pth'

# Output directory
OUTPUT_DIR = Path('notebooks/finetuned_porous')
OUTPUT_DIR.mkdir(exist_ok=True)

# Load config
with open(CONFIG_PATH, 'r') as f:
    CONFIG = json.load(f)

# Fine-tuning modifications
CONFIG['max_thickness'] = 1      # Reduced from 2
CONFIG['fill_ratio_min'] = 0.03  # Reduced from 0.05
CONFIG['fill_ratio_max'] = 0.10  # Reduced from 0.15
CONFIG['lr_finetune'] = 0.0005   # Half of original
CONFIG['batch_size'] = 4         # Adjust based on GPU memory

print('Config loaded with fine-tuning modifications:')
print(f"  max_thickness: {CONFIG['max_thickness']} (was 2)")
print(f"  lr_finetune: {CONFIG['lr_finetune']}")
print(f"  batch_size: {CONFIG['batch_size']}")

In [ ]:
# Load pretrained model
model = UrbanPavilionNCA(CONFIG).to(device)

checkpoint = torch.load(CHECKPOINT_PATH, map_location=device)
if isinstance(checkpoint, dict) and 'model_state_dict' in checkpoint:
    model.load_state_dict(checkpoint['model_state_dict'])
    print(f"Loaded checkpoint from epoch {checkpoint.get('epoch', 'unknown')}")
else:
    model.load_state_dict(checkpoint)
    print('Loaded checkpoint (raw state dict)')

n_params = sum(p.numel() for p in model.parameters())
print(f'Model parameters: {n_params:,}')
print(f'Model on: {next(model.parameters()).device}')

## 3. Define Loss Functions

In [ ]:
# ============ EXISTING LOSSES ============

class CorridorCoverageLoss(nn.Module):
    def __init__(self, config: dict):
        super().__init__()
        self.config = config

    def forward(self, structure, corridor_target):
        corridor_mask = corridor_target > 0.5
        corridor_volume = corridor_mask.float().sum() + 1e-8
        covered = (structure * corridor_mask.float()).sum()
        coverage_ratio = covered / corridor_volume
        return F.relu(0.7 - coverage_ratio)


class CorridorSpillLoss(nn.Module):
    def __init__(self, config: dict):
        super().__init__()
        self.config = config

    def forward(self, structure, corridor_target, legality_field):
        outside_corridor = (corridor_target < 0.5).float()
        legal_outside = outside_corridor * legality_field
        spill = (structure * legal_outside).sum()
        total_structure = structure.sum() + 1e-8
        return spill / total_structure


class GroundOpennessLoss(nn.Module):
    def __init__(self, config: dict):
        super().__init__()
        self.street_levels = config['street_levels']

    def forward(self, structure, corridor_target, legality_field):
        ground = structure[:, :self.street_levels]
        corridor_ground = corridor_target[:, :self.street_levels]
        outside_corridor_ground = (corridor_ground < 0.5).float()
        unwanted = ground * outside_corridor_ground
        return unwanted.mean()


class ThicknessLoss(nn.Module):
    def __init__(self, max_thickness: int = 1):
        super().__init__()
        self.max_thickness = max_thickness

    def forward(self, structure):
        binary = (structure > 0.5).float()
        eroded = binary.clone()
        for _ in range(self.max_thickness):
            eroded = -F.max_pool3d(-eroded.unsqueeze(0).unsqueeze(0), 3, 1, 1).squeeze()
        core_ratio = eroded.sum() / (binary.sum() + 1e-8)
        return core_ratio


class SparsityLossV31(nn.Module):
    def __init__(self, target_ratio: float = 0.06):
        super().__init__()
        self.target = target_ratio

    def forward(self, structure, available):
        available_vol = available.sum() + 1e-8
        filled = structure.sum()
        ratio = filled / available_vol
        excess = F.relu(ratio - self.target)
        return excess ** 2


class DensityPenalty(nn.Module):
    def __init__(self, target: float = 0.04):
        super().__init__()
        self.target = target

    def forward(self, structure):
        return F.relu(structure.mean() - self.target)


class TotalVariation3D(nn.Module):
    def forward(self, x):
        dz = torch.abs(x[:, 1:, :, :] - x[:, :-1, :, :]).mean()
        dy = torch.abs(x[:, :, 1:, :] - x[:, :, :-1, :]).mean()
        dx = torch.abs(x[:, :, :, 1:] - x[:, :, :, :-1]).mean()
        return (dz + dy + dx) / 3


class AccessConnectivityLoss(nn.Module):
    def __init__(self, config: dict):
        super().__init__()
        self.config = config

    def forward(self, state):
        cfg = self.config
        structure = state[:, cfg['ch_structure']]
        access = state[:, cfg['ch_access']]
        dilated = F.max_pool3d(structure.unsqueeze(1), 3, 1, 1).squeeze(1)
        connected = (access * dilated).sum()
        total_access = access.sum() + 1e-8
        return 1.0 - (connected / total_access)


class LoadPathLoss(nn.Module):
    def __init__(self, config: dict):
        super().__init__()
        self.config = config

    def forward(self, state):
        cfg = self.config
        structure = state[:, cfg['ch_structure']]
        B, D, H, W = structure.shape
        ground_support = structure[:, 0:1].clone()
        supported = ground_support
        for z in range(1, D):
            below = F.max_pool3d(supported.unsqueeze(1), (1, 3, 3), 1, (0, 1, 1)).squeeze(1)
            supported = torch.max(supported, torch.min(structure[:, z:z+1], below[:, -1:]))
        unsupported = F.relu(structure - supported.expand_as(structure))
        return unsupported.mean()


class CantileverLoss(nn.Module):
    def forward(self, structure):
        below = F.pad(structure[:, :-1], (0, 0, 0, 0, 1, 0))
        unsupported = F.relu(structure - below)
        return unsupported.mean()


class FacadeContactLoss(nn.Module):
    def __init__(self, config: dict, max_contact_ratio: float = 0.15):
        super().__init__()
        self.config = config
        self.max_ratio = max_contact_ratio

    def forward(self, state):
        cfg = self.config
        structure = state[:, cfg['ch_structure']]
        existing = state[:, cfg['ch_existing']]
        dilated_existing = F.max_pool3d(existing.unsqueeze(1), 3, 1, 1).squeeze(1)
        facade_zone = dilated_existing * (1 - existing)
        contact = (structure * facade_zone).sum()
        total_struct = structure.sum() + 1e-8
        ratio = contact / total_struct
        return F.relu(ratio - self.max_ratio)


print('Existing losses defined')

In [ ]:
# ============ NEW POROSITY LOSSES ============

class PorosityLoss(nn.Module):
    """
    Encourage internal voids within the structure.
    Penalizes structures where interior voxels are filled.
    """
    def __init__(self, target_porosity: float = 0.25, erosion_steps: int = 1):
        super().__init__()
        self.target_porosity = target_porosity
        self.erosion_steps = erosion_steps

    def forward(self, structure, corridor):
        binary = (structure > 0.5).float()
        eroded = binary.clone()
        for _ in range(self.erosion_steps):
            eroded = -F.max_pool3d(
                -eroded.unsqueeze(0).unsqueeze(0), 3, 1, 1
            ).squeeze(0).squeeze(0)

        interior_mask = eroded > 0.5
        if interior_mask.sum() < 10:
            return torch.tensor(0.0, device=structure.device)

        interior_filled = (structure * interior_mask.float()).sum()
        interior_total = interior_mask.float().sum()
        fill_ratio = interior_filled / (interior_total + 1e-8)
        porosity = 1.0 - fill_ratio
        return F.relu(self.target_porosity - porosity)


class SurfaceAreaLoss(nn.Module):
    """
    Encourage higher surface-to-volume ratio.
    Higher = more articulated/lattice-like.
    """
    def __init__(self, target_ratio: float = 2.5):
        super().__init__()
        self.target_ratio = target_ratio

    def forward(self, structure):
        binary = (structure > 0.5).float()
        volume = binary.sum() + 1e-8
        padded = F.pad(binary.unsqueeze(0).unsqueeze(0), (1,1,1,1,1,1), value=0).squeeze()

        surface = 0
        surface += torch.abs(padded[1:, :, :] - padded[:-1, :, :]).sum()
        surface += torch.abs(padded[:, 1:, :] - padded[:, :-1, :]).sum()
        surface += torch.abs(padded[:, :, 1:] - padded[:, :, :-1]).sum()

        ratio = surface / volume
        return F.relu(self.target_ratio - ratio)


print('New porosity losses defined')
print(f'  PorosityLoss: target_porosity=0.25')
print(f'  SurfaceAreaLoss: target_ratio=2.5')

## 4. Fine-Tuning Trainer

In [ ]:
class PorosityFineTuner:
    def __init__(self, model, config, device):
        self.model = model
        self.config = config
        self.device = device
        cfg = config

        max_thickness = cfg.get('max_thickness', 1)
        max_facade = cfg.get('max_facade_contact', 0.15)

        # Loss functions
        self.legality_loss = LocalLegalityLoss(cfg)
        self.coverage_loss = CorridorCoverageLoss(cfg)
        self.spill_loss = CorridorSpillLoss(cfg)
        self.ground_loss = GroundOpennessLoss(cfg)
        self.thickness_loss = ThicknessLoss(max_thickness)
        self.sparsity_loss = SparsityLossV31(target_ratio=0.06)
        self.facade_loss = FacadeContactLoss(cfg, max_facade)
        self.access_conn_loss = AccessConnectivityLoss(cfg)
        self.loadpath_loss = LoadPathLoss(cfg)
        self.cantilever_loss = CantileverLoss()
        self.density_loss = DensityPenalty(target=0.04)
        self.tv_loss = TotalVariation3D()
        self.porosity_loss = PorosityLoss(target_porosity=0.25)
        self.surface_loss = SurfaceAreaLoss(target_ratio=2.5)

        # MODIFIED WEIGHTS for porosity
        self.weights = {
            'legality': 30.0,
            'coverage': 20.0,
            'spill': 25.0,
            'ground': 35.0,
            'thickness': 50.0,    # INCREASED
            'sparsity': 45.0,     # INCREASED
            'facade': 10.0,
            'access_conn': 15.0,
            'loadpath': 10.0,
            'cantilever': 5.0,
            'density': 8.0,       # INCREASED
            'tv': 0.5,            # REDUCED
            'porosity': 20.0,     # NEW
            'surface': 10.0,      # NEW
        }

        self.optimizer = torch.optim.Adam(
            model.parameters(),
            lr=cfg.get('lr_finetune', 0.0005)
        )
        self.scene_gen = UrbanSceneGenerator(cfg)
        self.history = []

    def _random_scene_params(self):
        G = self.config['grid_size']
        gap_width = np.random.randint(8, 16)
        gap_center = G // 2
        left_x = (0, gap_center - gap_width // 2)
        right_x = (gap_center + gap_width // 2, G)

        buildings = [
            {'x': list(left_x), 'y': [0, np.random.randint(20, 28)],
             'z': [0, np.random.randint(10, 18)], 'side': 'left', 'gap_facing_x': left_x[1]},
            {'x': list(right_x), 'y': [0, np.random.randint(16, 24)],
             'z': [0, np.random.randint(10, 18)], 'side': 'right', 'gap_facing_x': right_x[0]}
        ]

        access_points = [
            {'x': gap_center, 'y': np.random.randint(18, 26), 'z': 0, 'type': 'ground'}
        ]
        if np.random.random() > 0.3:
            access_points.append({
                'x': np.random.choice([left_x[1], right_x[0]]),
                'y': np.random.randint(2, 8),
                'z': np.random.randint(6, 12),
                'type': 'elevated'
            })

        return {'buildings': buildings, 'access_points': access_points}

    def train_epoch(self, epoch):
        self.model.train()
        cfg = self.config
        batch_size = cfg.get('batch_size', 4)

        seeds, corridors = [], []
        for _ in range(batch_size):
            params = self._random_scene_params()
            seed, _ = self.scene_gen.generate(params, device=self.device)
            corridor = compute_corridor_target_v31(seed, cfg)
            seeds.append(seed)
            corridors.append(corridor)

        seeds = torch.cat(seeds, dim=0)
        corridor_target = torch.cat(corridors, dim=0)

        seed_scale = cfg.get('corridor_seed_scale', 0.15)
        if seed_scale > 0:
            seeds[:, cfg['ch_structure']] = torch.clamp(
                seeds[:, cfg['ch_structure']] + seed_scale * corridor_target, 0, 1
            )

        self.optimizer.zero_grad()
        steps = np.random.randint(40, 60)
        final = self.model(seeds, steps=steps)

        structure = final[:, cfg['ch_structure']]
        existing = final[:, cfg['ch_existing']]
        available = 1.0 - existing
        legality_field = self.legality_loss.compute_legality_field(final)

        # Compute all losses
        losses = {
            'legality': self.legality_loss(final),
            'coverage': self.coverage_loss(structure, corridor_target),
            'spill': self.spill_loss(structure, corridor_target, legality_field),
            'ground': self.ground_loss(structure, corridor_target, legality_field),
            'thickness': self.thickness_loss(structure),
            'sparsity': self.sparsity_loss(structure, available),
            'facade': self.facade_loss(final),
            'access_conn': self.access_conn_loss(final),
            'loadpath': self.loadpath_loss(final),
            'cantilever': self.cantilever_loss(structure),
            'density': self.density_loss(structure),
            'tv': self.tv_loss(structure),
            'porosity': self.porosity_loss(structure, corridor_target),
            'surface': self.surface_loss(structure),
        }

        total_loss = sum(self.weights[k] * v for k, v in losses.items())

        total_loss.backward()
        torch.nn.utils.clip_grad_norm_(self.model.parameters(), cfg.get('grad_clip', 1.0))
        self.optimizer.step()

        metrics = {
            'epoch': epoch,
            'total_loss': total_loss.item(),
            'L_porosity': losses['porosity'].item(),
            'L_thickness': losses['thickness'].item(),
            'L_coverage': losses['coverage'].item(),
            'L_sparsity': losses['sparsity'].item(),
            'fill_ratio': structure.mean().item(),
        }
        self.history.append(metrics)
        return metrics

    def evaluate(self, n_samples=8):
        self.model.eval()
        cfg = self.config
        results = []

        with torch.no_grad():
            for _ in range(n_samples):
                params = self._random_scene_params()
                seed, _ = self.scene_gen.generate(params, device=self.device)
                corridor = compute_corridor_target_v31(seed, cfg)

                seed_scale = cfg.get('corridor_seed_scale', 0.15)
                if seed_scale > 0:
                    seed[:, cfg['ch_structure']] += seed_scale * corridor

                grown = self.model(seed, steps=50)
                structure = grown[:, cfg['ch_structure']]

                corridor_mask = corridor > 0.5
                coverage = (structure * corridor_mask.float()).sum() / (corridor_mask.sum() + 1e-8)
                outside = (corridor < 0.5).float()
                spill = (structure * outside).sum() / (structure.sum() + 1e-8)

                results.append({
                    'coverage': coverage.item(),
                    'spill': spill.item(),
                    'fill_ratio': structure.mean().item(),
                })

        return {
            'avg_coverage': np.mean([r['coverage'] for r in results]),
            'avg_spill': np.mean([r['spill'] for r in results]),
            'avg_fill_ratio': np.mean([r['fill_ratio'] for r in results]),
        }

    def save_checkpoint(self, path, epoch):
        torch.save({
            'epoch': epoch,
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'config': self.config,
            'weights': self.weights,
            'history': self.history,
        }, path)
        print(f'Saved checkpoint to {path}')


print('PorosityFineTuner defined')

In [ ]:
# Create trainer
trainer = PorosityFineTuner(model, CONFIG, device)

print('Fine-tuning weights:')
for name, weight in trainer.weights.items():
    marker = ' *' if name in ['thickness', 'sparsity', 'density', 'porosity', 'surface'] else ''
    print(f'  {name}: {weight}{marker}')
print('\n* = modified/new for porosity')

## 5. Run Fine-Tuning

In [ ]:
# Fine-tuning parameters
N_EPOCHS = 150       # Total epochs
EVAL_EVERY = 10      # Evaluate every N epochs
SAVE_EVERY = 25      # Save checkpoint every N epochs

print(f'Starting fine-tuning for {N_EPOCHS} epochs...')
print(f'  Device: {device}')
print(f'  Batch size: {CONFIG["batch_size"]}')
print(f'  Learning rate: {CONFIG["lr_finetune"]}')
print(f'  Eval every {EVAL_EVERY} epochs')
print(f'  Save every {SAVE_EVERY} epochs')
print(f'  Output: {OUTPUT_DIR}')

In [ ]:
# Training loop
from tqdm.notebook import tqdm

best_fill_ratio = 1.0

for epoch in tqdm(range(1, N_EPOCHS + 1), desc='Fine-tuning'):
    metrics = trainer.train_epoch(epoch)

    if epoch % 5 == 0:
        print(
            f"Epoch {epoch:4d} | Loss: {metrics['total_loss']:.2f} | "
            f"Fill: {metrics['fill_ratio']*100:.1f}% | "
            f"Poros: {metrics['L_porosity']:.3f} | "
            f"Thick: {metrics['L_thickness']:.3f}"
        )

    if epoch % EVAL_EVERY == 0:
        eval_results = trainer.evaluate(n_samples=8)
        print(f"  [EVAL] Coverage: {eval_results['avg_coverage']*100:.1f}% | "
              f"Spill: {eval_results['avg_spill']*100:.1f}% | "
              f"Fill: {eval_results['avg_fill_ratio']*100:.1f}%")

        if eval_results['avg_fill_ratio'] < best_fill_ratio and eval_results['avg_coverage'] > 0.5:
            best_fill_ratio = eval_results['avg_fill_ratio']
            trainer.save_checkpoint(OUTPUT_DIR / 'best_porous.pth', epoch)

    if epoch % SAVE_EVERY == 0:
        trainer.save_checkpoint(OUTPUT_DIR / f'checkpoint_epoch_{epoch}.pth', epoch)

# Final save
trainer.save_checkpoint(OUTPUT_DIR / 'final_porous.pth', N_EPOCHS)
print('\nFine-tuning complete!')

## 6. Visualize Results

In [ ]:
# Plot training history
history = trainer.history
epochs = [h['epoch'] for h in history]

fig, axes = plt.subplots(2, 3, figsize=(14, 8))

axes[0, 0].plot(epochs, [h['total_loss'] for h in history])
axes[0, 0].set_title('Total Loss')
axes[0, 0].set_xlabel('Epoch')

axes[0, 1].plot(epochs, [h['L_porosity'] for h in history], 'g')
axes[0, 1].set_title('Porosity Loss')
axes[0, 1].set_xlabel('Epoch')

axes[0, 2].plot(epochs, [h['L_thickness'] for h in history], 'purple')
axes[0, 2].set_title('Thickness Loss')
axes[0, 2].set_xlabel('Epoch')

axes[1, 0].plot(epochs, [h['fill_ratio'] * 100 for h in history], 'orange')
axes[1, 0].set_title('Fill Ratio %')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].axhline(y=10, color='r', linestyle='--', alpha=0.5, label='Target max')

axes[1, 1].plot(epochs, [h['L_coverage'] for h in history], 'b')
axes[1, 1].set_title('Coverage Loss')
axes[1, 1].set_xlabel('Epoch')

axes[1, 2].plot(epochs, [h['L_sparsity'] for h in history], 'r')
axes[1, 2].set_title('Sparsity Loss')
axes[1, 2].set_xlabel('Epoch')

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'training_curves.png', dpi=150)
plt.show()

In [ ]:
# Test the fine-tuned model
model.eval()

torch.manual_seed(42)
np.random.seed(42)

scene_gen = UrbanSceneGenerator(CONFIG)
params = {
    'buildings': [
        {'x': [1, 9], 'y': [0, 27], 'z': [0, 14], 'side': 'left', 'gap_facing_x': 9},
        {'x': [23, 32], 'y': [0, 19], 'z': [0, 14], 'side': 'right', 'gap_facing_x': 23}
    ],
    'access_points': [
        {'x': 12, 'y': 23, 'z': 0, 'type': 'ground'},
        {'x': 9, 'y': 1, 'z': 11, 'type': 'elevated'}
    ]
}

seed_state, _ = scene_gen.generate(params, device=device)
corridor = compute_corridor_target_v31(seed_state, CONFIG)

seed_scale = CONFIG.get('corridor_seed_scale', 0.15)
if seed_scale > 0:
    seed_state[:, CONFIG['ch_structure']] += seed_scale * corridor

with torch.no_grad():
    result = model(seed_state, steps=50)

structure = (result[0, CONFIG['ch_structure']] > 0.5).cpu().numpy()
print(f'Fine-tuned model fill ratio: {structure.mean()*100:.1f}%')
print(f'Structure voxels: {structure.sum()}')

## 7. Export Model

In [ ]:
# Copy best model to deploy folder
import shutil

deploy_model_dir = Path('deploy/models')
deploy_model_dir.mkdir(exist_ok=True)

best_checkpoint = OUTPUT_DIR / 'best_porous.pth'
if best_checkpoint.exists():
    shutil.copy(best_checkpoint, deploy_model_dir / 'v31_porous.pth')
    print(f'Copied best model to {deploy_model_dir / "v31_porous.pth"}')
else:
    final_checkpoint = OUTPUT_DIR / 'final_porous.pth'
    shutil.copy(final_checkpoint, deploy_model_dir / 'v31_porous.pth')
    print(f'Copied final model to {deploy_model_dir / "v31_porous.pth"}')

# Save config
with open(deploy_model_dir / 'config_porous.json', 'w') as f:
    json.dump(CONFIG, f, indent=2)
print(f'Saved config to {deploy_model_dir / "config_porous.json"}')

In [ ]:
# Download the model (for local use)
from google.colab import files

print('Downloading fine-tuned model...')
files.download(str(deploy_model_dir / 'v31_porous.pth'))
files.download(str(deploy_model_dir / 'config_porous.json'))

## Summary

Fine-tuning complete! The model has been trained with:

1. **Increased thickness penalty** (30 -> 50)
2. **Increased sparsity penalty** (30 -> 45)
3. **New PorosityLoss** targeting 25% internal voids
4. **New SurfaceAreaLoss** encouraging articulated forms
5. **Reduced max_thickness** (2 -> 1)

### Next Steps
1. Download the model files
2. Place `v31_porous.pth` and `config_porous.json` in your local `deploy/models/` folder
3. Update `server.py` to load the new model (or add a model selector)